In [ ]:
import requests
import json
import pickle

## [abo | gun | blm]
campaign = 'blm'

url_tweet_cascade = 'http://130.56.248.5:9357/active-tweets/_search?pretty&scroll=1m'
url_tweet_cascade_scroll = 'http://130.56.248.5:9357/_search/scroll?pretty'

with open('queries/{}_tweet.json'.format(campaign), 'r') as f:
    query_temp = f.read()

query_json = json.loads(query_temp)
query_str = json.dumps(query_json)

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

tweets = []
scroll_ids = []
total = 0

r = requests.post(url_tweet_cascade, data=query_str, headers=headers)
result = json.loads(r.content.decode('utf-8'))
hits = result['hits']['hits']
print("# returned hits:", len(hits))
num_total_hits = result['hits']['total']
print("# total hits:", num_total_hits)
print("max_score:", result['hits']['max_score'])
print("=============================================")
last_scroll_id = result['_scroll_id']
scroll_ids.append(last_scroll_id)
total += len(hits)

for hit in hits:
    tweets.append(hit)

## Scroll search
while len(hits) > 0:
    try:
        scroll_query_json = {
            "scroll" : "1m", 
            "scroll_id" : last_scroll_id
        }
        
        scroll_query_str = json.dumps(scroll_query_json)
        r = requests.post(url_tweet_cascade_scroll, data=scroll_query_str, headers=headers)
        result = json.loads(r.content.decode('utf-8'))
        hits = result['hits']['hits']
        print("# returned hits:", len(hits))
        total += len(hits)
        print("# total:", total)
        last_scroll_id = result['_scroll_id']
        scroll_ids.append(last_scroll_id)
        for hit in hits:
            tweets.append(hit)
    except:
        print('--- Error ---')
        print(result)
        break


print("# tweets retrieved:", len(tweets))
print("# total:", total)

results = {}
for tweet in tweets:
    tweetId = tweet['_source']['tweet_id_str']
    results[tweetId] = tweet


try:
    pickle.dump(results, open("../data/social_media/{}/tweets_by_keywords.pkl".format(campaign), "wb"))
except:
    print("couldn't dump the tweets!!!")
    pass

## Clear/Close the sessions
clear_scroll_query_json = {"scroll_id": scroll_ids}
clear_scroll_query_str = json.dumps(clear_scroll_query_json)
r = requests.delete(url_tweet_cascade_scroll, data=clear_scroll_query_str, headers=headers)
